In [4]:
import os 
import string,re
import time
from multiprocessing import Pool
import comFuncs
from comFuncs import *
import ebooklib.epub as epub

In [5]:
f="/media/some/sophie-hhd/books/thrill/Rom-The_Aristocrat_and_the_Single_Mom_-_Michelle_Douglas.epub"

In [6]:
def getMeta(f):
    m=!exiftool "$f"
    meta={}
    for mm in m:
        tmp=mm.split(" : ")
        if len(tmp)==2:
            meta[tmp[0].strip()] = tmp[1].strip()
    fp,fn=os.path.split(f)
    n, ext = os.path.splitext(fn)
    return [n,meta]

In [19]:
fp,fn=os.path.split(f)
n, ext = os.path.splitext(fn)
n=n.replace("_"," ")
re.sub("^[A-Z][a-z][a-z][a-z]?-","",n)

'The Aristocrat and the Single Mom - Michelle Douglas'

In [20]:
def getAT(f):
    #f=f.encode('ascii','ignore')
    fp,ff= os.path.split(f)
    fnn, fext = os.path.splitext(ff)
    fn=re.sub(" \(v[0-9].[0-9]\)","",fnn)
    fn=re.sub("\(([e|m|l][p|o|i][u|b|t][b|i]?).*\)","",fn)
    fn=re.sub("\[([H|S][P|D]).*\]","",fn)
    fn=re.sub("^[A-Z][a-z][a-z][a-z]?-","",fn)
    fn=fn.replace("_"," ")
    #print fn
    seri=''
    sm=0
    mtd=None
    sp=fn.split()
    if len(sp)>4:
        tit=sp[-2:]
        auth=sp[:-2]
    elif len(sp) >1:
        tit=fn.split()[-1:]
        auth=fn.split()[:-1]
    else:
        return "no meta"
    
    if fn.count('-')==1:
        tit,auth=fn.split('-')
    elif fn.count('-')==2:
        seri,tit,auth=fn.split('-')
        if tit.count("[")>0:
            tm=seri
            seri=tit.replace('[',"").replace("]","")
            tit=tm
        
    ##print "title",tit,"author",auth
   
    ''' if tit.count('_ ')>0:
        spp=tit.split('_')[0]
        ##print spp
        if spp.count(' ')>0:
            tit=' '.join(spp.split())
        else:
            tit=spp'''
    '''if auth.count('_ ')>0:
        spp=auth.split('_')[0]
        ##print spp
        if spp.count(' ')>0:
            auth=' '.join(spp.split())
        else:
            auth=spp'''
    if seri.count(" ")>0:
        seri=" ".join(seri.split())
    if tit.count(' ')>0:
        title=''.join(l for l in ' '.join(tit.split()) if l in string.printable)
    else:
        title=''.join(l for l in tit if l in string.printable)
    if auth.count(" ")>0:    
        author=''.join(l for l in ' '.join(auth.split()) if l in string.printable)
    else:
        author=''.join(l for l in auth if l in string.ascii_letters+" .")
    ret=[fn,f]
    
    if len(author.split())>0:
        ret.append(author)
    if len(title.split())>0:
        ret.append(title)
    if seri!='' and len(seri.split())>0:
        ret.append(seri)
    #print(len(ret))
    if len(ret)>2:
        return ret

In [ ]:
def getATfM(f):
    metakey='http://purl.org/dc/elements/1.1/'
    fp,ff= os.path.split(f)
    fnn, fext = os.path.splitext(ff)
    fn=re.sub(" \(v[0-9].[0-9]\)","",fnn)
    fn=re.sub("\(([e|m|l][p|o|i][u|b|t][b|i]?).*\)","",fn)
    try:
        book=epub.read_epub(f)
    except:
        return getAT(f)
    ret=[fn,f]
    if 'creator' in book.metadata[metakey].keys() and len(book.metadata[metakey]['creator'][0])>0:
        ret.append(book.metadata[metakey]['creator'][0][0])
    if 'title' in book.metadata[metakey].keys() and len(book.metadata[metakey]['title'])>0 :
        ret.append(book.metadata[metakey]['title'][0][0])
    if len(ret)>2:
        return ret
    else:
        return getAT(f)